# Word Embedding

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
import gensim
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import to_categorical
import pickle

nltk.download('punkt')

SEQUENCE_LENGTH = 100
EMBEDDING_SIZE = 100
BATCH_SIZE = 64
PAD_WORD = "__PAD__"
WORD_EMBEDDING_FILE = 'word_embedding.csv'
TOKENIZER_FILE = 'tokenizer.pickle'


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

ModuleNotFoundError: No module named 'gensim'

In [ ]:
data_word_embedding = pd.read_csv("dota2_chat_final.csv", encoding="latin_1", usecols=["match", "slot", "text"])
data_word_embedding.head(10)

In [ ]:
data_word_embedding_numpy = data_word_embedding.drop(columns=['match', 'slot']).to_numpy().flatten()

In [ ]:
# document to sentences
word_embedding_sentences = []
for index, document in enumerate(data_word_embedding_numpy):
    sentences = sent_tokenize(document);
    for index, sentence in enumerate(sentences):
        word_embedding_sentences.append(sentence)    
word_embedding_sentences

In [ ]:
# melakukan tokenisasi (menghilangkan tanda baca, dll)
# membuat dictionary setiap token
word_embedding_tokenizer = Tokenizer(oov_token='OOV')
word_embedding_tokenizer.fit_on_texts(word_embedding_sentences)
word_embedding_tokenizer.word_index

In [ ]:
# encoding setiap token sesuai dengan nomor indexnya
word_embedding_sequences = word_embedding_tokenizer.texts_to_sequences(word_embedding_sentences)
word_embedding_sequences

In [ ]:
# mengembalikan sequence index kembali katanya yang sudah hilang tanda bacanya
# karena input gensim adalah kata
data_word_embedding_numpy_temp = [[word_embedding_tokenizer.index_word[word_index] for word_index in sentence] for sentence in word_embedding_sequences]
data_word_embedding_numpy_temp

In [ ]:
# training word embedding
word_embedding_model = gensim.models.Word2Vec(data_word_embedding_numpy_temp, min_count = 1, size = EMBEDDING_SIZE, window = 5, sg = 1) 
word_index = word_embedding_tokenizer.word_index
# membuat array yang berisi value word_embedding berdasar index katanya pada dictionary
word_embedding_temp = np.zeros((len(word_index)+1, EMBEDDING_SIZE))
for word, word_object in word_embedding_model.wv.vocab.items():
    index = word_index[word]
    word_embedding_temp[index] = word_embedding_model.wv[word]    
    
# store tokenizer dictionary 
with open(TOKENIZER_FILE, 'wb') as handle:
    pickle.dump(word_embedding_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# word embedding to fil
np.savetxt(WORD_EMBEDDING_FILE, word_embedding_temp, delimiter=',')

In [ ]:
word_embedding_temp[tokenizer.word_index['gg']]